In [ ]:
# pltting

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import csv
from cmath import nan

In [ ]:
# set things
proteins = ["mcl1", "p38", "tyk2"]

In [ ]:
prot_dict = {}

for prot in proteins:

    with open(f"{prot}_solvated_molecules_no_nan.csv", "w") as file:
        writer = csv.writer(file, delimiter=";")
        for line in open(f"{prot}_solvated_molecules.csv"):
            parts = line.split(";")
            if parts[5] != "nan":
                writer.writerow([parts[0],parts[1],parts[2],parts[3],parts[4],parts[5],parts[6],parts[7].replace("\n","")])
            else:
                continue
                # print(line)

    df = pd.read_csv(f"{prot}_solvated_molecules_no_nan.csv", header="infer", delimiter=";")

    box_shapes = ["cubic","truncatedoctahedron"]
    length_list = [10,15,20,25,30,35,40]
    legs = ["lig","sys"]
    plotting_dict = {"box": [], "length": [], "legs": [], "avg_mols": [], "std_mols": []}

    for box in box_shapes:
        for length in length_list:
            for leg in legs:
                values = df[df["boxlength"] == length][df["boxtype"] == box][df["leg"] == leg]["nMolecules"].to_numpy()
                avg_mols = np.mean(values)
                std_mols = np.std(values)
                plotting_dict["box"].append(box)
                plotting_dict["length"].append(length)
                plotting_dict["legs"].append(leg)
                plotting_dict["avg_mols"].append(avg_mols)
                plotting_dict["std_mols"].append(std_mols)

    df2 = pd.DataFrame(data=plotting_dict)
    prot_dict[prot] = df2


In [ ]:
box_shapes = ["cubic","truncatedoctahedron"]
length_list = [10,15,20,25,30,35,40]
legs = ["lig","sys"]
plotting_dict = {"box": [], "length": [], "legs": [], "avg_mols": [], "std_mols": []}

for box in box_shapes:
    for length in length_list:
        for leg in legs:
            values = []
            for prot in proteins:
                df = prot_dict[prot]
                value = df.loc[df["length"] == length][df["box"] == box][df["legs"] == leg]["avg_mols"].item()
                values.append(value)

            avg_mols = np.average(values)
            std_mols = np.std(values)

            plotting_dict["box"].append(box)
            plotting_dict["length"].append(length)
            plotting_dict["legs"].append(leg)
            plotting_dict["avg_mols"].append(avg_mols)
            plotting_dict["std_mols"].append(std_mols)

mean_df = pd.DataFrame(data=plotting_dict)
prot_dict["mean"] = mean_df
mean_df

In [ ]:
plt.figure(figsize=(10,10))

protein = "mean"
df2 = prot_dict[protein]

colour_list = ["orange","navy","hotpink","teal"]
col_index = 0 
for box in box_shapes:
        for leg in legs:
            plt.plot(df2[df2["legs"] == leg][df2["box"] == box]["length"], df2[df2["legs"] == leg][df2["box"] == box]["avg_mols"],
            label=f"{box},{leg}",
            color=colour_list[col_index]
            )

            # plot error bars:
            yerr = df2[df2["legs"] == leg][df2["box"] == box]["std_mols"]

            plt.errorbar(df2[df2["legs"] == leg][df2["box"] == box]["length"], df2[df2["legs"] == leg][df2["box"] == box]["avg_mols"], 
                        yerr=yerr,
                        ls="none",
                        color=colour_list[col_index],
                        )
            
            col_index += 1

plt.ylabel("average nMolecules()")
plt.xlabel("Size of box (Angstrom)")
plt.title(f"{protein} solvation")
plt.legend()

plt.savefig(f"{protein}_solvation.png", dpi=300)

plt.show()